# Summarize datasets for all operator results
Code author: Sahar H. El Abbadi
Date started: 2023-03-13
Date last edited: 2023-03-17

## Generate summary tables

Generate overpass summary tables for each operator. Overpass summary tables extract all relevant information from operator reports and metering data that is needed for downstream analysis. These tables are saved as CSV files in 03_results > overpass_summary, using the format operator_stage_overpasses.csv.

Columns in the overpass summary tables are:
- overpass_id: this is the ID for the specific aircraft overpass, and matches PerformerExperimentID overpass number in the raw operator data
- overpass_datetime: date and time of overpass, in UTC time. This can be generated using the Flightradar GPS timestamp (timekeeper = 'flightradar'), Stanford's on the ground estimate of overhead time (timekeeper = 'stanford'), or using the timestamp according to the operator report (timekeeper = 'team'). Value used for all analysis is in paper is Flightradar
- zero_release: True if the release by Stanford is 0 kgh, False if greater than 0 kgh
- non_zero_release: True if release by Stanford is greater than 0 kgh, False if equal to 0 kgh
- operator_kept: True if this overpass passed the operator QC criteria
- stanford_kept: True if this overpass passed Stanford's QC criteria
- phase_iii: True if we provided this overpass to the operator during Phase III of unblinding
- pass_all_qc: True if passed both operator and Stanford QC
- fail_all_qc: True if this overpass failed both operator and Stanford QC
- operator_detected: True if operator detected a release. False if they did not
- operator_quantification: operator's quantification estimate as reported in operator report
- operator_lower: lower bound on operator's lower bound quantification estimate
- operator_upper: upper bound on operator's quantification estimate
- qc_summary: summarizes results of both operator and Stanford QC. Must be one of the following: 'pass_all', 'fail_stanford', 'fail_operator', 'fail_all'

Note: due to the small number of data points for Scientific Aviation and the fact that they did not submit using a template, sciav_1_overpasses.csv was manually constructed by Sahar El Abbadi on 2023-03-16.

In [ ]:
# Imports
from methods_source import generate_all_overpass_reports

In [ ]:

generate_all_overpass_reports(strict_discard=False, timekeeper='flightradar', gas_comp_source='km', time_ave='60')
generate_all_overpass_reports(strict_discard=True, timekeeper='flightradar', gas_comp_source='km', time_ave='60')

## Sanity checks on data

Basic double checks on the data to make sure everything looks good

##### Number of overpasses

Check that the length of the dataframe matches the highest overpass ID. This will flag the potential for missing or duplicate data in the summary file. Doing this check as we previously observed duplicate rows in the meter summary file.

In [ ]:
# Check Carbon Mapper

from methods_source import load_overpass_summary, check_overpass_number

cm_overpasses = load_overpass_summary(operator='Carbon Mapper', stage=1, strict_discard=False, time_ave=60, gas_comp_source='km')
cm_max_id = cm_overpasses.max()['overpass_id']
cm_overpasses_length = len(cm_overpasses)
check_overpass_number(operator='Carbon Mapper', max_overpass_id=cm_max_id, overpasses_length=cm_overpasses_length)

In [ ]:
# Check GHGSat data length

from methods_source import load_overpass_summary, check_overpass_number

ghg_overpasses = load_overpass_summary(operator='GHGSat', stage=1, strict_discard=False, time_ave=60, gas_comp_source='km')
ghg_max_id = ghg_overpasses.max()['overpass_id']
ghg_overpasses_length = len(ghg_overpasses)
check_overpass_number(operator='GHGSat', max_overpass_id=ghg_max_id, overpasses_length=ghg_overpasses_length)

In [ ]:
# Check Kairos data length

from methods_source import load_overpass_summary, check_overpass_number

kairos_overpasses = load_overpass_summary(operator='Kairos', stage=1, strict_discard=False, time_ave=60, gas_comp_source='km')
kairos_max_id = kairos_overpasses.max()['overpass_id']
kairos_overpasses_length = len(kairos_overpasses)
check_overpass_number(operator='Kairos', max_overpass_id=kairos_max_id, overpasses_length=kairos_overpasses_length)

In [ ]:
# Check Scientific Aviation data length

from methods_source import load_overpass_summary, check_overpass_number

sciav_overpasses = load_overpass_summary(operator='Scientific Aviation', stage=1, strict_discard=False,time_ave=60, gas_comp_source='km')
sciav_max_id = sciav_overpasses.max()['overpass_id']
sciav_overpasses_length = len(sciav_overpasses)
check_overpass_number(operator='Scientific Aviation', max_overpass_id=sciav_max_id, overpasses_length=sciav_overpasses_length)

In [ ]:
# Check Methane Air

from methods_source import load_overpass_summary, check_overpass_number

mair_overpasses = load_overpass_summary(operator='Methane Air', stage=1, strict_discard=False, time_ave=60, gas_comp_source='km')
mair_max_id = mair_overpasses.max()['overpass_id']
mair_overpasses_length = len(mair_overpasses)
check_overpass_number(operator='Methane Air', max_overpass_id=mair_max_id, overpasses_length=mair_overpasses_length)

##### Calculate average flight altitude across all reported overpasses

Use "altitude_feet" column in overpass summary files

In [ ]:
from methods_source import load_overpass_summary

cm_overpasses = load_overpass_summary(operator='Carbon Mapper', stage=1, strict_discard=False, time_ave=60, gas_comp_source='km')
ghg_overpasses = load_overpass_summary(operator='GHGSat', stage=1, strict_discard=False, time_ave=60, gas_comp_source='km')
kairos_overpasses = load_overpass_summary(operator='Kairos', stage=1, strict_discard=False, time_ave=60, gas_comp_source='km')
mair_overpasses = load_overpass_summary(operator='Methane Air', stage=1, strict_discard=False, time_ave=60, gas_comp_source='km')
cm_altitude = cm_overpasses.altitude_feet.mean()
ghg_altitude = ghg_overpasses.altitude_feet.mean()
kairos_altitude = kairos_overpasses.altitude_feet.mean()
mair_altitude = mair_overpasses.altitude_feet.mean()
print('Average overpass altitude (feet) across all reported overpasses:\n')
print(f'Carbon Mapper: {cm_altitude:,.1f} ft')
print(f'GHGSat: {ghg_altitude:,.1f} ft')
print(f'Kairos: {kairos_altitude:,.1f} ft')
print(f'Methane Air: {mair_altitude:,.1f} ft')

##### Calculate average flight for each day of testing


In [ ]:
from methods_source import calc_daily_altitude
operator = 'Carbon Mapper'
cm_alt = calc_daily_altitude(operator)
print(f'{operator} average daily altitude:\n')
print(cm_alt.to_string(formatters={'altitude_feet':'{:,.1f} feet'.format}))

In [ ]:
operator = 'GHGSat'
ghg_alt = calc_daily_altitude(operator)
print(f'{operator} average daily altitude:\n')
print(ghg_alt.to_string(formatters={'altitude_feet':'{:,.1f} feet'.format}))

In [ ]:
operator = 'Kairos'
kairos_alt = calc_daily_altitude(operator)
print(f'{operator} average daily altitude:\n')
print(kairos_alt.to_string(formatters={'altitude_feet':'{:,.1f} feet'.format}))

In [ ]:
operator = 'Methane Air'
mair_alt = calc_daily_altitude(operator)
print(f'{operator} average daily altitude:\n')
print(mair_alt.to_string(formatters={'altitude_feet':'{:,.1f} feet'.format}))

##### Summarize distribution of overpasses

Total number of overpasses, missing data, max and min given to each team, number that passed Stanford QC

Missing overpasses: documented as an overpass by Stanford, but not included in Operator Report file

In [ ]:
from methods_source import find_missing_data, load_overpass_summary
# Carbon Mapper
operator = 'Carbon Mapper'
missing = find_missing_data(operator)
print(f'{operator} missing overpasses: {len(missing)}')

overpasses = load_overpass_summary(operator, stage=1)
max_release_cols = ['release_rate_kgh', 'lower_95CI', 'upper_95CI', 'ch4_kgh_sigma', 'sigma_flow_variability', 'sigma_meter_reading', 'sigma_gas_composition']
max_info = overpasses.loc[overpasses.release_rate_kgh.idxmax()][max_release_cols]
print(f'Largest volume overpass for {operator}:')
print(f'Release Rate: {max_info.release_rate_kgh:} kg CH4 / hr')
print(f'[{max_info.lower_95CI}, {max_info.upper_95CI}, 95% CI]')
print(f'(sigma from gas flow: {max_info.sigma_flow_variability:1f})')
print(f'(sigma from meter: {max_info.sigma_meter_reading:1f})')
print(f'(sigma from gas composition: {max_info.sigma_gas_composition:1f})')
print(f'(combined total sigma: {max_info.ch4_kgh_sigma:1f})')





In [ ]:
print(overpasses)